In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/anime.csv')

In [ ]:
df.info()

In [ ]:
df.drop(labels=['Name',"English name","Japanese name",'Aired','Producers','Licensors','Duration','Members','Watching','On-Hold'], axis='columns', inplace=True)

In [ ]:
df.info()

In [ ]:
print(df["Score"])

In [ ]:
print(df["Episodes"])

In [ ]:
numeric_cols = [
    'Score', 'Episodes', 'Ranked',
    'Score-10','Score-9','Score-8','Score-7','Score-6',
    'Score-5','Score-4','Score-3','Score-2','Score-1'
]

In [ ]:
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [ ]:
print(df[numeric_cols].isna().sum())

In [ ]:
df["Episodes"] =df['Episodes'].fillna(0)

In [ ]:
score_cols = ['Score-10','Score-9','Score-8','Score-7','Score-6',
              'Score-5','Score-4','Score-3','Score-2','Score-1']
df[score_cols] = df[score_cols].fillna(0)

In [ ]:
print(df[numeric_cols].isna().sum())

In [ ]:
df.dropna(subset="Score", inplace=True)

In [ ]:
print(df[numeric_cols].isna().sum())

In [ ]:
freq = df["Studios"].value_counts().count()
print("Printing the frequency")
display(freq)

In [ ]:
cat = df.select_dtypes(include='O').keys()

cat

In [ ]:
# Get dummy variables for each unique genre
genre_dummies = df['Genres'].str.get_dummies(sep=', ')

# Merge with original dataframe
df = pd.concat([df, genre_dummies], axis=1)

In [ ]:
df.info()

In [ ]:
df[['Season','Year']] = df['Premiered'].str.split(' ', expand=True)
df.drop(columns=['Season'], inplace=True)
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

In [ ]:
df.info()

In [ ]:
df['Year'].fillna(0, inplace=True)

In [ ]:
freq = df["Type"].value_counts()
print("Printing the frequency")
display(freq)

In [ ]:
freq = df['Rating'].value_counts()
display(freq)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
categorical_columns = ['Type', 'Rating']

In [ ]:
encoder = OneHotEncoder(categories='auto')

In [ ]:
onehot_encoded = encoder.fit_transform(df[categorical_columns]).toarray()

In [ ]:
one_hot_df = pd.DataFrame(onehot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

In [ ]:
display(one_hot_df)

In [ ]:
df.drop(labels=['Genres'], axis='columns', inplace=True)

 3   Type           12421 non-null  object 

 6   Studios        12421 non-null  object 
 

8   Rating         12421 non-null  object 




In [ ]:
freq = df['Rating'].value_counts()

In [ ]:
display(freq)

In [ ]:
type_dummies = pd.get_dummies(df['Type'], prefix='Type')
df = pd.concat([df, type_dummies], axis=1)

In [ ]:
df.info()

In [ ]:
df[['Rating_index','Rating_info']] = df['Rating'].str.split(' - ', expand=True)
df.drop(columns=['Rating_info'], inplace=True)

rating_dummies = pd.get_dummies(df['Rating_index'], prefix='Rating')
df = pd.concat([df, rating_dummies], axis=1)

In [ ]:
df.head()

In [ ]:
df.drop(labels=['Rating_index'], axis='columns', inplace=True)
df.info()

In [ ]:
df.drop(labels=['Rating_Unknown'], axis='columns', inplace=True)

In [ ]:
top_studios = df['Studios'].value_counts().nlargest(10).index
df['Studios'] = df['Studios'].apply(lambda x: x if x in top_studios else 'Other')


In [ ]:
display(df['Studios'].value_counts().head(10))

In [ ]:
studio_dummies = pd.get_dummies(df['Studios'], prefix='Studio')
df = pd.concat([df, studio_dummies], axis=1)

In [ ]:
df.info()

In [ ]:
df.drop(labels=['Studios', 'Studio_Unknown', 'Type', 'Rating'], axis='columns', inplace=True)

In [ ]:
df.describe()

In [ ]:
bool_cols = df.select_dtypes('bool').columns

df[bool_cols] = df[bool_cols].astype(int)

In [ ]:
X = df.drop(columns=['Score', 'MAL_ID'])
score_related_cols = ['Score-10','Score-9','Score-8','Score-7',
                      'Score-6','Score-5','Score-4','Score-3','Score-2','Score-1', 'Ranked']

X = X.drop(columns=score_related_cols)
y = df['Score']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

model = RandomForestRegressor(
    n_estimators=200, random_state=42, n_jobs=-1
)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))


In [ ]:
X.info()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

importances = pd.Series(model.feature_importances_, index=X.columns)
importances.nlargest(20).plot(kind='barh', figsize=(8,6))
plt.title("Top-20 Feature Importances")
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

scores = cross_val_score(model, X, y, cv=5, scoring='r2', n_jobs=-1)
print("R2:", scores)
print("Mean R2:", np.mean(scores))